In [1]:
import numpy as np
import math
from random import seed
from random import randrange
from random import random

In [2]:
# computing the signals for every layer
def signal(weights, input_vector):
    signal = weights[0]
    signal += np.dot(weights[1::], input_vector[1::])
    return signal

# transformation function
def transformation_function(signal):
    return np.tanh(signal)
           
#Forward Propagation
def forward_propagate(network_weights, input_vector):
    print("FORWARD PROPAGATION:")
    network_inputs = [input_vector]
    
    
    for layer in network_weights:
        new_inputs = []
        for neuron in layer:
            s = signal(neuron, input_vector)
            out = transformation_function(s)
            new_inputs.append(out)
        input_vector = np.concatenate(([1],np.array(new_inputs)))
        network_inputs.append(np.array(input_vector))
        
    return np.array(new_inputs),np.array(network_inputs)

In [10]:
# computing theta'
def activation_derivative(output):
    return np.fromiter([(1 - output[i]**2) for i in range(1, len(output))], float)

def back_propagate(network_layers, network_inputs, network_weights, y_true, y_pred):
    print("BACK PROPAGATION:")
    delta = np.empty_like(network_weights)
    for i,j in zip(range(len(network_weights)-1,-1,-1), range(len(network_inputs)-1,-1,-1)):
        if(i == network_layers-1):
            theta_dash =  activation_derivative(network_inputs[j])
            error = np.array(y_pred[1::] - y_true[1::])
            delta[i] = np.array([2 * error.dot(theta_dash)])
        
        else:
            for neuron in network_weights[i+1]:
                theta_dash =  activation_derivative(network_inputs[j])
                w,b = neuron[1:],neuron[0]
                error_mul = w.reshape(len(w),1).dot(delta[-1])
                delt_mul = error_mul * theta_dash
            delta[i] = delt_mul
            
    return delta

In [13]:
def compute_gradient(n_inputs, network_layers, network_inputs, y_true, y_pred, delta):
    print("GRADIENTS:")
    
    e_in = (np.array(y_pred[1::]) - np.array(y_true[1::]))**2
    
    gradient = list()
    for i in range(1, len(network_inputs)):
        delt = delta[i].reshape(len(delta[i]),1)
        outs = network_inputs[i-1].reshape(len(network_inputs[i-1]),1)
        gradient.append(delt.dot(outs.transpose()))
    
    return e_in, np.array(gradient)

In [25]:
n_inputs = 2
network_layers = 2
n_neurons = 5
iterations = 1
seed(1)

input_layer = np.array([1, 2, 1])
hidden_layer = np.random.rand(n_neurons, n_inputs + 1)
output_layer = np.random.rand(1,n_neurons+1)
network_weights = np.array((hidden_layer,output_layer))

y_true = [None, 1]
print("NETWORK SETUP:")
print("x(0) inputs = ", input_layer)
print("network weights = ", network_weights)
print("expected = ", y_true[1::])
print()

y_pred, network_inputs = forward_propagate(network_weights, input_layer)
y_pred = [None]+[i for i in y_pred]
print("predicted = ", y_pred[1::])
print("network inputs x(l) = ",network_inputs)
print()

delta = back_propagate(network_layers, network_inputs, network_weights, np.array(y_true), np.array(y_pred))
delta = [None]+[i for i in delta]
print("delta = ", delta[1::])
print()

e_in, gradient = compute_gradient(n_inputs, network_layers, network_inputs, y_true, y_pred, np.array(delta))
print("in-sample error = ", e_in)
print("gradient = ", gradient)

NETWORK SETUP:
x(0) inputs =  [1 2 1]
network weights =  [array([[0.71160684, 0.48037473, 0.91375538],
       [0.19605092, 0.15554568, 0.89875072],
       [0.95037506, 0.64734497, 0.54981098],
       [0.32431807, 0.9468681 , 0.92762557],
       [0.88543808, 0.41110204, 0.10730174]])
 array([[0.02029026, 0.5103036 , 0.46197613, 0.11369882, 0.70930847,
        0.9477371 ]])]
expected =  [1]

FORWARD PROPAGATION:
predicted =  [0.9901204520276361]
network inputs x(l) =  [array([1, 2, 1])
 array([1.        , 0.98872009, 0.88661881, 0.9925559 , 0.99630237,
       0.94833188])
 array([1.        , 0.99012045])]

BACK PROPAGATION:
delta =  [array([-4.44724840e-06, -3.83908936e-05, -6.55182153e-07, -2.03408223e-06,
       -3.70644031e-05]), array([-0.00038849])]

GRADIENTS:
in-sample error =  [9.76054681e-05]
gradient =  [array([[-4.44724840e-06, -8.89449680e-06, -4.44724840e-06],
       [-3.83908936e-05, -7.67817871e-05, -3.83908936e-05],
       [-6.55182153e-07, -1.31036431e-06, -6.55182153e-0